# sodapy tutorial using NYC Open Data 
Mark Bauer

# Introduction  
This notebook demonstrates how to use sodapy, the python client for the Socrata Open Data API, with NYC Open Data. Examples of popular methods are included, as well as basic queries using SoQL, the Socrata Query Language. 

# Information about sodapy

## sodapy
sodapy is a python client for the Socrata Open Data API.

## To learn more about sodapy
Installing: https://pypi.org/project/sodapy/  
GitHub: https://github.com/xmunoz/sodapy

## The official Socrata Open Data API (SODA) docs
https://dev.socrata.com/

## Queries using SODA
https://dev.socrata.com/docs/queries/

## Inspiration for this notebook
https://github.com/xmunoz/sodapy/blob/master/examples/soql_queries.ipynb

# Importing Libraries

In [223]:
# importing libraries
import pandas as pd
from sodapy import Socrata
import itertools 

In [224]:
%reload_ext watermark

In [225]:
%watermark -a "Mark Bauer" -u -t -d -v -p pandas,sodapy,itertools

Mark Bauer 
last updated: 2021-01-23 16:48:36 

CPython 3.7.1
IPython 7.18.1

pandas 1.0.0
sodapy 2.0.0
itertools unknown


Documention for installing watermark: https://github.com/rasbt/watermark

# Using sodapy

In order for a user to use sodapy, they need to retrieve a source domain (i.e. the open data source you are trying to connect to). Additionally, if a user wants to query a specific data set, then the data set identifier (i.e. the data set id on the given source domain) needs to be identified as well. Below, we identify NYC Open Data's source domain: `data.cityofnewyork.us` and the data set identifier for the NYC 311 data set: `erm2-nwe9`.

![nyc-311-api-docs](nyc-311-api-docs.png)  

source: https://dev.socrata.com/foundry/data.cityofnewyork.us/erm2-nwe9

We save this information as variables below.

In [226]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

# Socrata class in sodapy

In [272]:
# The main class that interacts with the SODA API.

# The required arguments are:
#     domain: the domain you wish you to access
#     app_token: your Socrata application token
# Simple requests are possible without an app_token, though these
# requests will be rate-limited.

client = Socrata(socrata_domain, None, timeout=1000)

In [228]:
type(client)

sodapy.Socrata

Socrata Methods

![socrata-methods](socrata-methods.png)

source: https://github.com/xmunoz/sodapy#datasetslimit0-offset0

We will review a few of the popular methods in this tutorial.

# Socrata Methods

## `.datasets()`

`datasets` method: Returns the list of datasets associated with a particular domain.
WARNING: Large limits (>1000) will return megabytes of data,
which can be slow on low-bandwidth networks, and is also a lot of
data to hold in memory.

In [229]:
print(type(client.datasets()))

<class 'list'>


In [230]:
print(type(client.datasets()[0]))

<class 'dict'>


In [231]:
client.datasets()[0].keys()

dict_keys(['resource', 'classification', 'metadata', 'permalink', 'link', 'owner', 'creator'])

In [232]:
# Viewing the resource dictionary in the first item of the datasets list. From there, 
# we view the first five items under the resource dictionary.

limit = 5

dict(itertools.islice(client.datasets()[0]['resource'].items(), limit))

{'name': 'DOB Job Application Filings',
 'id': 'ic3t-wcy2',
 'parent_fxf': [],
 'description': 'This dataset contains all job applications submitted through the Borough Offices, through eFiling, or through the HUB, which have a "Latest Action Date" since January 1, 2000. This dataset does not include jobs submitted through DOB NOW. See the DOB NOW: Build – Job Application Filings dataset for DOB NOW jobs.',
 'attribution': 'Department of Buildings (DOB)'}

In [233]:
# Once we've identified the structure of the dictionary, we try to 
# find the 311 data set and identify its position in the datasets list.

idx = 0

for dataset_id in client.datasets():
    if client.datasets()[idx]['resource']['id'] == 'erm2-nwe9':
        print(client.datasets()[idx]['resource']['name'], \
              '\nindex:', idx)
        break
    else:
        idx += 1    

311 Service Requests from 2010 to Present 
index: 5


In [234]:
# Previewing information about the 311 data set from the datasets method.
# Note: Information is quite long.

idx_311 = idx
print(idx_311)

client.datasets()[idx_311]

5


{'resource': {'name': '311 Service Requests from 2010 to Present',
  'id': 'erm2-nwe9',
  'parent_fxf': [],
  'description': '<b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>\r\n\r\nAll 311 Service Requests from 2010 to present. This information is automatically updated daily.',
  'attribution': '311, DoITT',
  'attribution_link': None,
  'contact_email': None,
  'type': 'dataset',
  'updatedAt': '2021-01-23T02:33:09.000Z',
  'createdAt': '2011-10-10T05:52:17.000Z',
  'metadata_updated_at': '2020-04-22T20:18:38.000Z',
  'data_updated_at': '2021-01-23T02:33:09.000Z',
  'page_views': {'page_views_last_week': 1237,
   'page_views_last_month': 4749,
   'page_views_total': 437885,
   'page_views_last_week_log': 10.273795599214266,
   'p

In [235]:
# Since the datasets method is long, let's see if we can identify specific keys we want to preview
# in the resource dictionary.

lst_of_keys = list(client.datasets()[idx_311]['resource'].keys())

print(lst_of_keys)

['name', 'id', 'parent_fxf', 'description', 'attribution', 'attribution_link', 'contact_email', 'type', 'updatedAt', 'createdAt', 'metadata_updated_at', 'data_updated_at', 'page_views', 'columns_name', 'columns_field_name', 'columns_datatype', 'columns_description', 'columns_format', 'download_count', 'provenance', 'lens_view_type', 'blob_mime_type', 'hide_from_data_json', 'publication_date']


In [236]:
# Previewing keys, values in the resource dictionary.

for item in lst_of_keys:
    print(item + ':', client.datasets()[idx_311]['resource'][item], '\n')

name: 311 Service Requests from 2010 to Present 

id: erm2-nwe9 

parent_fxf: [] 

description: <b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>

All 311 Service Requests from 2010 to present. This information is automatically updated daily. 

attribution: 311, DoITT 

attribution_link: None 

contact_email: None 

type: dataset 

updatedAt: 2021-01-23T02:33:09.000Z 

createdAt: 2011-10-10T05:52:17.000Z 

metadata_updated_at: 2020-04-22T20:18:38.000Z 

data_updated_at: 2021-01-23T02:33:09.000Z 

page_views: {'page_views_last_week': 1237, 'page_views_last_month': 4749, 'page_views_total': 437885, 'page_views_last_week_log': 10.273795599214266, 'page_views_last_month_log': 12.213711798105672, 'page_views_total_log': 18.74019579941932

lens_view_type: tabular 

blob_mime_type: None 

hide_from_data_json: False 

publication_date: 2018-04-20T03:03:18.000Z 



## `.get()`

`get` method: Read data from the requested resource. Options for content_type are json,
csv, and xml.

In [273]:
# Using try and except statements because these requests are large and may timeout.
# If the request timesout, we skip it. 

try:
    print(type(client.get(socrata_dataset_identifier)))
except:
    print('timeout error. skipping.')
    pass  

<class 'list'>


In [274]:
# Using try and except statements because these requests are large and may timeout.
# If the request timesout, we skip it. 

try:
    print(client.get(socrata_dataset_identifier)[0].keys())
except:
    print('timeout error. skipping.')
    pass

dict_keys(['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'location_type', 'incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2', 'intersection_street_1', 'intersection_street_2', 'city', 'landmark', 'status', 'community_board', 'bbl', 'borough', 'x_coordinate_state_plane', 'y_coordinate_state_plane', 'open_data_channel_type', 'park_facility_name', 'park_borough', 'latitude', 'longitude', 'location', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih'])


In [275]:
# Using try and except statements because these requests are large and may timeout.
# If the request timesout, we skip it. 

try:
    print(client.get(socrata_dataset_identifier, select='*')[0])
except:
    print('timeout error. skipping.')
    pass

{'unique_key': '49585185', 'created_date': '2021-01-22T02:07:52.000', 'agency': 'NYPD', 'agency_name': 'New York City Police Department', 'complaint_type': 'Blocked Driveway', 'descriptor': 'Partial Access', 'location_type': 'Street/Sidewalk', 'incident_zip': '11218', 'incident_address': '475 EAST    8 STREET', 'street_name': 'EAST    8 STREET', 'cross_street_1': 'CORTELYOU ROAD', 'cross_street_2': 'DITMAS AVENUE', 'intersection_street_1': 'CORTELYOU ROAD', 'intersection_street_2': 'DITMAS AVENUE', 'city': 'BROOKLYN', 'landmark': 'EAST    8 STREET', 'status': 'In Progress', 'community_board': '12 BROOKLYN', 'bbl': '3053920085', 'borough': 'BROOKLYN', 'x_coordinate_state_plane': '992409', 'y_coordinate_state_plane': '171761', 'open_data_channel_type': 'MOBILE', 'park_facility_name': 'Unspecified', 'park_borough': 'BROOKLYN', 'latitude': '40.63811710562249', 'longitude': '-73.97060224522494', 'location': {'latitude': '40.63811710562249', 'longitude': '-73.97060224522494', 'human_address'

## `.get_metadata()`

`get_metadata` method: Retrieve the metadata for a particular dataset.

In [240]:
type(client.get_metadata(socrata_dataset_identifier))

dict

In [241]:
# Previewing keys in dictionary
client.get_metadata(socrata_dataset_identifier).keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'indexUpdatedAt', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowClass', 'rowIdentifierColumnId', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [242]:
# Previewing keys vertically.
keys = client.get_metadata(socrata_dataset_identifier).keys()
for key in keys:
    print(key)

id
name
assetType
attribution
averageRating
category
createdAt
description
displayType
downloadCount
hideFromCatalog
hideFromDataJson
indexUpdatedAt
newBackend
numberOfComments
oid
provenance
publicationAppendEnabled
publicationDate
publicationGroup
publicationStage
rowClass
rowIdentifierColumnId
rowsUpdatedAt
rowsUpdatedBy
tableId
totalTimesRated
viewCount
viewLastModified
viewType
approvals
columns
grants
metadata
owner
query
rights
tableAuthor
tags
flags


In [243]:
# Previewing the id and name of the data set.
print('id and name of dataset\n' + \
      '-' * 30 + \
      '\nid:', client.get_metadata(socrata_dataset_identifier)['id'], \
      '\nname:', client.get_metadata(socrata_dataset_identifier)['name'])

id and name of dataset
------------------------------
id: erm2-nwe9 
name: 311 Service Requests from 2010 to Present


In [244]:
# Previewing the first 30 keys, values of the dictionary.
metadata = client.get_metadata(socrata_dataset_identifier)

limit = 30
out = dict(itertools.islice(metadata.items(), limit))
print(type(out), '\n')

for key, value in out.items():
    print(key + ':',  value)

<class 'dict'> 

id: erm2-nwe9
name: 311 Service Requests from 2010 to Present
assetType: dataset
attribution: 311, DoITT
averageRating: 0
category: Social Services
createdAt: 1318225937
description: <b>NOTE: This data does not present a full picture of 311 calls or service requests, in part because of operational and system complexities associated with remote call taking necessitated by the unprecedented volume 311 is handling during the Covid-19 crisis. The City is working to address this issue. </b>

All 311 Service Requests from 2010 to present. This information is automatically updated daily.
displayType: table
downloadCount: 398179
hideFromCatalog: False
hideFromDataJson: False
indexUpdatedAt: 1571326778
newBackend: True
numberOfComments: 19
oid: 28506835
provenance: official
publicationAppendEnabled: False
publicationDate: 1524193398
publicationGroup: 244403
publicationStage: published
rowClass: 
rowIdentifierColumnId: 354922030
rowsUpdatedAt: 1611369189
rowsUpdatedBy: 5fuc-pqz2

In [245]:
# Saving metadata dictionary as 'metadata'
metadata = client.get_metadata(socrata_dataset_identifier)
print(type(metadata))

# Previewing the datatype of columns
print(type(metadata['columns']), ', length:', len(metadata['columns']))

# Previewing the field names for each element in our columns list
for x in metadata['columns']:
    print(x['fieldName'])

<class 'dict'>
<class 'list'> , length: 46
unique_key
created_date
closed_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
cross_street_1
cross_street_2
intersection_street_1
intersection_street_2
address_type
city
landmark
facility_type
status
due_date
resolution_description
resolution_action_updated_date
community_board
bbl
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
vehicle_type
taxi_company_borough
taxi_pick_up_location
bridge_highway_name
bridge_highway_direction
road_ramp
bridge_highway_segment
latitude
longitude
location
:@computed_region_efsh_h5xi
:@computed_region_f5dn_yrer
:@computed_region_yeji_bk3q
:@computed_region_92fq_4b7q
:@computed_region_sbqj_enih


In [246]:
metadata = client.get_metadata(socrata_dataset_identifier)

# Previewing the first element in our columns list
metadata['columns'][0]

{'id': 354922030,
 'name': 'Unique Key',
 'dataTypeName': 'text',
 'description': 'Unique identifier of a Service Request (SR) in the open data set\n',
 'fieldName': 'unique_key',
 'position': 1,
 'renderTypeName': 'text',
 'tableColumnId': 1567787,
 'width': 220,
 'cachedContents': {'largest': '49593747',
  'non_null': '24784083',
  'null': '0',
  'top': [{'item': '10693408', 'count': '1'},
   {'item': '10836749', 'count': '1'},
   {'item': '10836967', 'count': '1'},
   {'item': '11051177', 'count': '1'},
   {'item': '11413576', 'count': '1'},
   {'item': '11463895', 'count': '1'},
   {'item': '11463896', 'count': '1'},
   {'item': '11464334', 'count': '1'},
   {'item': '11464394', 'count': '1'},
   {'item': '11464467', 'count': '1'},
   {'item': '11464508', 'count': '1'},
   {'item': '11464509', 'count': '1'},
   {'item': '11464521', 'count': '1'},
   {'item': '11464567', 'count': '1'},
   {'item': '11464572', 'count': '1'},
   {'item': '11464639', 'count': '1'},
   {'item': '1146484

In [247]:
metadata = client.get_metadata(socrata_dataset_identifier)

# Identifying our columns list
cols = metadata['columns']

# Previewing first position in our columns list
cols[1]

{'id': 354922031,
 'name': 'Created Date',
 'dataTypeName': 'calendar_date',
 'description': 'Date SR  was created\n',
 'fieldName': 'created_date',
 'position': 2,
 'renderTypeName': 'calendar_date',
 'tableColumnId': 1567788,
 'width': 244,
 'cachedContents': {'largest': '2021-01-22T02:07:52.000',
  'non_null': '24784083',
  'null': '0',
  'top': [{'item': '2013-01-24T00:00:00.000', 'count': '7650'},
   {'item': '2015-01-08T00:00:00.000', 'count': '7242'},
   {'item': '2014-01-07T00:00:00.000', 'count': '7030'},
   {'item': '2015-02-16T00:00:00.000', 'count': '6430'},
   {'item': '2014-01-08T00:00:00.000', 'count': '6197'},
   {'item': '2012-01-04T00:00:00.000', 'count': '5933'},
   {'item': '2013-11-25T00:00:00.000', 'count': '5909'},
   {'item': '2014-01-23T00:00:00.000', 'count': '5782'},
   {'item': '2014-01-22T00:00:00.000', 'count': '5497'},
   {'item': '2015-01-07T00:00:00.000', 'count': '5432'},
   {'item': '2011-01-24T00:00:00.000', 'count': '5380'},
   {'item': '2011-10-28T

In [248]:
# Creating a fieldName dictionary for every element in our column list
fieldName = {x['fieldName']: x for x in cols}

# Previewing the field names (values) in our fieldName dictionary
for key in fieldName.keys():
    print(key)

unique_key
created_date
closed_date
agency
agency_name
complaint_type
descriptor
location_type
incident_zip
incident_address
street_name
cross_street_1
cross_street_2
intersection_street_1
intersection_street_2
address_type
city
landmark
facility_type
status
due_date
resolution_description
resolution_action_updated_date
community_board
bbl
borough
x_coordinate_state_plane
y_coordinate_state_plane
open_data_channel_type
park_facility_name
park_borough
vehicle_type
taxi_company_borough
taxi_pick_up_location
bridge_highway_name
bridge_highway_direction
road_ramp
bridge_highway_segment
latitude
longitude
location
:@computed_region_efsh_h5xi
:@computed_region_f5dn_yrer
:@computed_region_yeji_bk3q
:@computed_region_92fq_4b7q
:@computed_region_sbqj_enih


In [249]:
# Removing the last five field names
list(fieldName.keys())[:-5]

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location']

In [250]:
# Removing the last five field names
cols_as_list = list(fieldName.keys())[:-5]

cols_as_list

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location']

# Socrata Query Language (SoQL)

## Analyzing NYC 311 Complaints

In [251]:
client = Socrata("data.cityofnewyork.us", None, timeout=1000)

query = """
SELECT 
    complaint_type, count(complaint_type)
GROUP BY 
    complaint_type
ORDER BY 
    count(complaint_type) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("erm2-nwe9", 
                     query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(10)

shape of data: (445, 2)


,complaint_type,count_complaint_type
0,Noise - Residential,2233988
1,HEAT/HOT WATER,1416620
2,Illegal Parking,1122944
3,Blocked Driveway,1044233
4,Street Condition,1020245
5,Street Light Condition,983232
6,HEATING,887869
7,PLUMBING,747234
8,Water System,686693
9,Noise - Street/Sidewalk,682019


In [269]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor)
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(10)

shape of data: (1000, 2)


,descriptor,count_descriptor
0,Loud Music/Party,2333531
1,ENTIRE BUILDING,922492
2,HEAT,871935
3,No Access,777499
4,Street Light Out,728990
5,Pothole,617428
6,Banging/Pounding,611857
7,APARTMENT ONLY,494128
8,CEILING,358690
9,Loud Talking,358647


In [276]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    date_trunc_ymd(created_date) AS day, count(day) AS count
GROUP BY 
    day
ORDER BY 
    count DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(10)

shape of data: (1000, 2)


,day,count
0,2020-08-04T00:00:00.000,23314
1,2020-08-05T00:00:00.000,18305
2,2020-07-05T00:00:00.000,16014
3,2020-07-04T00:00:00.000,15365
4,2020-06-20T00:00:00.000,15098
5,2020-06-21T00:00:00.000,14965
6,2020-06-28T00:00:00.000,12899
7,2020-06-27T00:00:00.000,12074
8,2020-08-09T00:00:00.000,12057
9,2020-08-06T00:00:00.000,12043


In [277]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    created_date as timestamp, date_trunc_ymd(created_date) as day, count(created_date) AS count
GROUP BY 
    timestamp
ORDER BY 
    count ASC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(10)

shape of data: (1000, 3)


,timestamp,day,count
0,2010-01-01T15:48:17.000,2010-01-01T00:00:00.000,1
1,2010-01-01T16:01:57.000,2010-01-01T00:00:00.000,1
2,2010-01-01T15:40:55.000,2010-01-01T00:00:00.000,1
3,2010-01-01T15:48:01.000,2010-01-01T00:00:00.000,1
4,2010-01-01T15:57:07.000,2010-01-01T00:00:00.000,1
5,2010-01-01T16:01:43.000,2010-01-01T00:00:00.000,1
6,2010-01-01T15:35:00.000,2010-01-01T00:00:00.000,1
7,2010-01-01T15:39:32.000,2010-01-01T00:00:00.000,1
8,2010-01-01T15:45:00.000,2010-01-01T00:00:00.000,1
9,2010-01-01T15:48:00.000,2010-01-01T00:00:00.000,1


## Analyzing NYC 311 Street Flooding Complaints

In [279]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    complaint_type, count(complaint_type) AS count
WHERE 
    LOWER(complaint_type) LIKE '%flood%'
GROUP BY 
    complaint_type
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (0, 0)


""


In [280]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    complaint_type, count(complaint_type) AS count
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    complaint_type
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (4, 2)


,complaint_type,count
0,Sewer,120245
1,Street Light Condition,9045
2,OEM Literature Request,271
3,Public Toilet,48


In [281]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor)
WHERE 
    complaint_type='Sewer'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
LIMIT 1000
"""

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.head(len(results_df))

shape of data: (27, 2)


,descriptor,count_descriptor
0,Sewer Backup (Use Comments) (SA),148553
1,Catch Basin Clogged/Flooding (Use Comments) (SC),89821
2,Catch Basin Sunken/Damaged/Raised (SC1),28514
3,Street Flooding (SJ),27512
4,Manhole Cover Broken/Making Noise (SB),19778
5,Manhole Cover Missing (Emergency) (SA3),17427
6,Sewer Odor (SA2),15339
7,Defective/Missing Curb Piece (SC4),8484
8,Manhole Overflow (Use Comments) (SA1),6832
9,Catch Basin Search (SC2),4153


In [282]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor) AS count
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df

shape of data: (11, 2)


,descriptor,count
0,Catch Basin Clogged/Flooding (Use Comments) (SC),89821
1,Street Flooding (SJ),27512
2,Flood Light Lamp Out,5962
3,Highway Flooding (SH),2834
4,Flood Light Lamp Cycling,2511
5,Ready NY - Flooding,271
6,Flood Light Lamp Dayburning,205
7,Flood Light Lamp Missing,190
8,Flood Light Lamp Dim,177
9,RAIN GARDEN FLOODING (SRGFLD),78


In [283]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    date_trunc_ymd(created_date) as day, count(created_date) AS count
WHERE 
    descriptor == 'Street Flooding (SJ)'
GROUP BY 
    day
ORDER BY 
    count DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)

results_df.head(10)

shape of data: (1000, 2)


,day,count
0,2017-05-05T00:00:00.000,247
1,2014-12-09T00:00:00.000,226
2,2014-04-30T00:00:00.000,189
3,2018-04-16T00:00:00.000,163
4,2013-05-08T00:00:00.000,162
5,2016-11-15T00:00:00.000,151
6,2016-02-08T00:00:00.000,150
7,2018-11-25T00:00:00.000,142
8,2020-07-10T00:00:00.000,131
9,2010-10-01T00:00:00.000,130


In [284]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

client = Socrata(socrata_domain, None, timeout=1000)

query = """
SELECT 
    *
WHERE 
    descriptor == 'Street Flooding (SJ)'
ORDER BY 
    created_date DESC
LIMIT 
    1000
"""

# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(socrata_dataset_identifier, query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print('shape of data:', results_df.shape)
results_df.to_csv('sample_data_street_flooding.csv')

results_df.head()

shape of data: (1000, 30)


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,...,park_facility_name,park_borough,latitude,longitude,location,closed_date,resolution_description,resolution_action_updated_date,intersection_street_1,intersection_street_2
0,49585453,2021-01-21T22:38:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11211,204 WILSON STREET,WILSON STREET,LEE AVE,...,Unspecified,BROOKLYN,40.70694739790969,-73.96001109794196,"{'latitude': '40.70694739790969', 'longitude':...",NaN,NaN,NaN,NaN,NaN
1,49590021,2021-01-21T19:33:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11427,88-01 LYMAN STREET,LYMAN STREET,88 AVE,...,Unspecified,QUEENS,40.73012063798075,-73.73178196964847,"{'latitude': '40.73012063798075', 'longitude':...",NaN,NaN,NaN,NaN,NaN
2,49586240,2021-01-21T13:06:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11420,121-08 111 AVENUE,111 AVENUE,121 ST,...,Unspecified,QUEENS,40.68105018373826,-73.81937878576748,"{'latitude': '40.68105018373826', 'longitude':...",NaN,NaN,NaN,NaN,NaN
3,49586973,2021-01-21T11:44:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11372,34-32 70 STREET,70 STREET,34 AVE,...,Unspecified,QUEENS,40.75164968710312,-73.89603558046888,"{'latitude': '40.75164968710312', 'longitude':...",NaN,NaN,NaN,NaN,NaN
4,49576602,2021-01-20T15:31:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11413,225 STREET,225 STREET,131 AVENUE,...,Unspecified,QUEENS,40.68004946200496,-73.74340162464055,"{'latitude': '40.68004946200496', 'longitude':...",2021-01-20T21:10:00.000,The Department of Environment Protection inspe...,2021-01-20T21:10:00.000,NaN,NaN


## Analyzing NYC 311 Data Sets with Most Downloads

In [285]:
type(client)

sodapy.Socrata

In [286]:
type(client.datasets())

list

In [287]:
len(client.datasets())

3140

In [288]:
df = pd.DataFrame.from_records(client.datasets())

df.head()

,resource,classification,metadata,permalink,link,owner,creator,preview_image_url
0,"{'name': 'DOB Job Application Filings', 'id': ...","{'categories': ['economy', 'environment', 'hou...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/ic3t-wcy2,https://data.cityofnewyork.us/Housing-Developm...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
1,"{'name': 'Civil Service List (Active)', 'id': ...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/vx8i-nprf,https://data.cityofnewyork.us/City-Government/...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
2,"{'name': 'TLC New Driver Application Status', ...","{'categories': ['transportation', 'environment...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/dpec-ucu7,https://data.cityofnewyork.us/Transportation/T...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
3,"{'name': 'For Hire Vehicles (FHV) - Active', '...","{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/8wbx-tsch,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN
4,{'name': 'For Hire Vehicles (FHV) - Active Dri...,"{'categories': [], 'tags': [], 'domain_categor...",{'domain': 'data.cityofnewyork.us'},https://data.cityofnewyork.us/d/xjfq-wh2d,https://data.cityofnewyork.us/Transportation/F...,"{'id': '5fuc-pqz2', 'user_type': 'interactive'...","{'id': '5fuc-pqz2', 'user_type': 'interactive'...",NaN


In [289]:
df = df.resource

df = pd.DataFrame.from_records(df)

df.head()

,name,id,parent_fxf,description,attribution,attribution_link,contact_email,type,updatedAt,createdAt,...,columns_field_name,columns_datatype,columns_description,columns_format,download_count,provenance,lens_view_type,blob_mime_type,hide_from_data_json,publication_date
0,DOB Job Application Filings,ic3t-wcy2,[],This dataset contains all job applications sub...,Department of Buildings (DOB),None,None,dataset,2021-01-23T21:10:07.000Z,2013-04-18T15:18:56.000Z,...,"[professional_cert, horizontal_enlrgmt, job_de...","[Text, Text, Text, Text, Text, Text, Text, Tex...",[Job is Professionally Certified by Licensed P...,"[{'align': 'right'}, {'align': 'right'}, {'ali...",36784.0,official,tabular,None,False,2020-06-22T18:23:35.000Z
1,Civil Service List (Active),vx8i-nprf,[],A Civil Service List consists of all candidate...,Department of Citywide Administrative Services...,None,None,dataset,2021-01-22T14:51:12.000Z,2016-06-14T21:12:15.000Z,...,"[exam_no, list_no, first_name, mi, last_name, ...","[text, number, text, text, text, number, text,...",[A four (4) digit number that identifies a civ...,"[{'displayStyle': 'plain', 'align': 'left'}, {...",34180.0,official,tabular,None,False,2021-01-22T14:51:12.000Z
2,TLC New Driver Application Status,dpec-ucu7,[],THIS DATASET IS UPDATED SEVERAL TIMES PER DAY....,Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-23T17:11:28.000Z,2016-05-17T18:43:43.000Z,...,"[defensive_driving, status, drug_test, lastupd...","[Text, Text, Text, Calendar date, Calendar dat...",[A NYS certified 6 hour Defensive Driving Cour...,"[{'displayStyle': 'plain', 'align': 'left'}, {...",NaN,official,tabular,None,False,2019-12-17T18:44:57.000Z
3,For Hire Vehicles (FHV) - Active,8wbx-tsch,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-23T20:32:55.000Z,2015-07-16T17:33:32.000Z,...,"[active, vehicle_license_number, name, license...","[text, text, text, text, calendar_date, text, ...","[Permit active or not\n, FHV Vehicle License N...","[{'displayStyle': 'plain', 'align': 'left'}, {...",253642.0,official,tabular,None,False,2021-01-23T20:32:55.000Z
4,For Hire Vehicles (FHV) - Active Drivers,xjfq-wh2d,[],"<b>PLEASE NOTE:</b> This dataset, which includ...",Taxi and Limousine Commission (TLC),None,None,dataset,2021-01-23T20:21:52.000Z,2015-07-16T17:24:02.000Z,...,"[license_number, name, type, expiration_date, ...","[number, text, text, calendar_date, text, cale...","[FHV License Number\n, Driver Name\n\n, Type o...","[{'precisionStyle': 'standard', 'noCommas': 't...",221593.0,official,tabular,None,False,2021-01-23T20:21:52.000Z


In [290]:
len(df)

3140

In [291]:
df[['name', 'download_count']].sort_values(by='download_count', ascending=False).head()

,name,download_count
33,Demographic Statistics By Zip Code,1013007.0
1238,Overhead Electronic Signs,429382.0
5,311 Service Requests from 2010 to Present,398174.0
10,Medallion Drivers - Active,287707.0
3,For Hire Vehicles (FHV) - Active,253642.0


In [294]:
highest_downloaded = df[['name', 'download_count']].sort_values(by='download_count', ascending=False)

print('The data set {}'.format(highest_downloaded['name'].iloc[0]), \
     'has {} downloads'.format(f"{highest_downloaded['download_count'].iloc[0]:,.0f}"), \
     'and is the most downloaded data set on NYC Open Data.')

The data set Demographic Statistics By Zip Code has 1,013,007 downloads and is the most downloaded data set on NYC Open Data.
